<a href="https://colab.research.google.com/github/GalinaZh/Appl_alg2021/blob/main/Applied_Alg_sem_8_repete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 8 
# Прикладная алгебра и численные методы
## Повторение: псевдообратная матрица, ортогонализация Грама-Шмидта, LU, QR.

In [2]:
!python -m pip install sympy --upgrade

Requirement already up-to-date: sympy in /usr/local/lib/python3.7/dist-packages (1.7.1)


In [3]:
import numpy as np
import sympy
from sympy import S, latex, Matrix, I, simplify, expand, GramSchmidt
import scipy.linalg
import numpy.linalg
from sympy.functions.special.polynomials import chebyshevt, chebyshevu
from numpy.polynomial.chebyshev import chebinterpolate
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

In [4]:
import sympy
import numpy
sympy.__version__, numpy.__version__

('1.7.1', '1.19.5')

In [5]:
sympy.init_printing(use_unicode=False, wrap_line=False, no_global=True)

## Псевдообратная матрица
В numpy псевдообратная матрица находится с помощью np.linalg.pinv
## Пример 1
Найти псевдообратную матрицу к матрице
$$
\left(
\begin{matrix}
1 & 4 & 7 & 10 & 13\\
16 & 19 & 22 & 25 & 28\\
31 & 34 & 37 & 40 & 43\\
46 & 49 & 52 & 55 & 58
\end{matrix}
\right)
$$

In [6]:
m = 4
n = 5
A = np.arange(1, 3*m*n + 1, 3).reshape((m, n))
As = Matrix(A)
As_pinv = As.pinv()
display(As, As_pinv, As_pinv.evalf(8))
for item in (np.linalg.pinv(A), scipy.linalg.pinv(A)):
    print('A^+ = {0},\n A = A A^+ A  {1}, A^+ = A^+ A ^+ {2}'.format(item, np.allclose(A, np.dot(A, np.dot(item, A))), np.allclose(item, np.dot(item, np.dot(A, item)))))

[1   4   7   10  13]
[                  ]
[16  19  22  25  28]
[                  ]
[31  34  37  40  43]
[                  ]
[46  49  52  55  58]

[-3/50   -7/225  -1/450    2/75 ]
[                               ]
[         -73               23  ]
[-4/125   ----   -1/2250   ---- ]
[         4500             1500 ]
[                               ]
[-1/250  -1/750   1/750   1/250 ]
[                               ]
[          61              -11  ]
[3/125    ----   7/2250    ---- ]
[         4500             1500 ]
[                               ]
[  13      32      11           ]
[ ---     ----    ----    -7/375]
[ 250     1125    2250          ]

[-0.06   -0.031111111   -0.0022222222    0.026666667 ]
[                                                    ]
[-0.032  -0.016222222   -0.00044444444   0.015333333 ]
[                                                    ]
[-0.004  -0.0013333333   0.0013333333       0.004    ]
[                                                    ]
[0.024    0.013555556    0.0031111111   -0.0073333333]
[                                                    ]
[0.052    0.028444444    0.0048888889   -0.018666667 ]

A^+ = [[-0.06       -0.03111111 -0.00222222  0.02666667]
 [-0.032      -0.01622222 -0.00044444  0.01533333]
 [-0.004      -0.00133333  0.00133333  0.004     ]
 [ 0.024       0.01355556  0.00311111 -0.00733333]
 [ 0.052       0.02844444  0.00488889 -0.01866667]],
 A = A A^+ A  True, A^+ = A^+ A ^+ True
A^+ = [[-0.06       -0.03111111 -0.00222222  0.02666667]
 [-0.032      -0.01622222 -0.00044444  0.01533333]
 [-0.004      -0.00133333  0.00133333  0.004     ]
 [ 0.024       0.01355556  0.00311111 -0.00733333]
 [ 0.052       0.02844444  0.00488889 -0.01866667]],
 A = A A^+ A  True, A^+ = A^+ A ^+ True


In [7]:
latex(As)

'\\left[\\begin{matrix}1 & 4 & 7 & 10 & 13\\\\16 & 19 & 22 & 25 & 28\\\\31 & 34 & 37 & 40 & 43\\\\46 & 49 & 52 & 55 & 58\\end{matrix}\\right]'

### LU разложение
$PA = LU$ для матрицы $A$

$L$ - левая треугольная матрица с единицами на главной диагонали,
$U$ - правая треугольная (трапециевидная) матрица,
$P$ - матрица перестановок.

$A = P^{-1}LU$.
### Пример 2.
Построим  LU разложение для матрицы 
$$
M=\left(
\begin{matrix}
0 & 3+ I & 5 - 2I\\
 - I & 2 & 1 - I\\
5 & -1 + 4I & -3
\end{matrix}
\right)
$$

In [8]:
M = Matrix([[0, 3 + I, 5 - 2*I], [-I, 2, 1 - I], [5, -1 + 4*I, -3]])
L, U, perm = M.LUdecomposition()
display('L', L, 'U', U, 'LU', simplify(expand(L*U)), 'M', M, 'perm', perm)

'L'

[ 1           0           0]
[                          ]
[ 0           1           0]
[                          ]
[     (-1 - 6*I)*(3 - I)   ]
[5*I  ------------------  1]
[             10           ]

'U'

[-I    2                         1 - I                     ]
[                                                          ]
[0   3 + I                      5 - 2*I                    ]
[                                                          ]
[                              (-1 - 6*I)*(3 - I)*(5 - 2*I)]
[0     0    -3 - 5*I*(1 - I) - ----------------------------]
[                                           10             ]

'LU'

[-I     2       1 - I ]
[                     ]
[0    3 + I    5 - 2*I]
[                     ]
[5   -1 + 4*I    -3   ]

'M'

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

'perm'

Произведение матриц LU отличается от исходной матрицы M перестановкой строк. Восстановим матрицу M, применяя перестановки в соответствии с результатом, выдаваемым LUdecomposition:

In [9]:
number_of_rows = M.shape[0]
L, U, perm = M.LUdecomposition()
LU = simplify(expand((L*U).permuteBkwd(perm)))
P = sympy.eye(number_of_rows).permuteFwd(perm)
display('L', L, 'U', U, 'perm', perm, 'P', P, 'LU', LU, 
        'M', M, 'PM == LU', P*M == simplify(expand(L*U))) 

'L'

[ 1           0           0]
[                          ]
[ 0           1           0]
[                          ]
[     (-1 - 6*I)*(3 - I)   ]
[5*I  ------------------  1]
[             10           ]

'U'

[-I    2                         1 - I                     ]
[                                                          ]
[0   3 + I                      5 - 2*I                    ]
[                                                          ]
[                              (-1 - 6*I)*(3 - I)*(5 - 2*I)]
[0     0    -3 - 5*I*(1 - I) - ----------------------------]
[                                           10             ]

'perm'

...

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

'M'

[0    3 + I    5 - 2*I]
[                     ]
[-I     2       1 - I ]
[                     ]
[5   -1 + 4*I    -3   ]

'PM == LU'

True

### QR разложение
$A = QR$ 

$Q$ - матрица из ортогональных столбцов, т.е. $Q^HQ = I$, $I$ - единичная матрица, 
$R$ - правая треугольная (трапециевидная) матрица.

Ранг матрицы $A$ равен числу столбцов матрицы $Q$.
### Пример 3.
Построим  QR разложение для матрицы 

$$
M_3 = \left(
\begin{matrix}
0 & 3  & 5 & 6 \\
 - 1 & 2 & 1 & -7\\
5 & -1  & -3 & 8
\end{matrix}
\right)
$$

In [10]:
M3 = Matrix([[0, 3, 5, 6], [-1, 2, 1, -7], [5, -1, -3, 8]])
Q3, R3 = M3.QRdecomposition()
Q3R3 = simplify(expand(Q3*R3))
display('Q3', simplify(expand(Q3)), 'R3', simplify(expand(R3)), 
        'Q3R3', Q3R3, 'M3', M3, 'M3 = Q3R3', M3 == Q3R3)

'Q3'

[             _____       ____]
[           \/ 910    3*\/ 35 ]
[   0       -------   --------]
[              35        35   ]
[                             ]
[   ____       _____     ____ ]
[-\/ 26    3*\/ 910   -\/ 35  ]
[--------  ---------  --------]
[   26        182        7    ]
[                             ]
[    ____      _____     ____ ]
[5*\/ 26   3*\/ 910   -\/ 35  ]
[--------  ---------  --------]
[   26        910        35   ]

'R3'

[             ____        ____        ____ ]
[  ____  -7*\/ 26    -8*\/ 26    47*\/ 26  ]
[\/ 26   ----------  ----------  --------- ]
[            26          13          26    ]
[                                          ]
[            _____        _____       _____]
[        3*\/ 910    68*\/ 910   15*\/ 910 ]
[  0     ---------   ----------  ----------]
[            26         455         182    ]
[                                          ]
[                         ____        ____ ]
[                    13*\/ 35     9*\/ 35  ]
[  0         0       ---------    -------- ]
[                        35          7     ]

'Q3R3'

[0   3   5   6 ]
[              ]
[-1  2   1   -7]
[              ]
[5   -1  -3  8 ]

'M3'

[0   3   5   6 ]
[              ]
[-1  2   1   -7]
[              ]
[5   -1  -3  8 ]

'M3 = Q3R3'

True

 ## Псевдообратная матрица на основе QR разложения:
 $$
M = QR, \quad M^+ = R^T(R\cdot R^T)^{-1}Q^T
 $$

 ## Пример 4
 Воспользуемся этой формулой для построения псевдообратной матрицы Примера 3:

In [11]:
M3pinv = M3.pinv()
M3pinv.simplify()
display(M3pinv, R3.T*(R3*R3.T)**(-1)*Q3.T)

[       460    1195]
[1/61   ----   ----]
[       2013   6039]
[                  ]
[  17   431    631 ]
[ ---   ----   ----]
[ 183   2013   6039]
[                  ]
[              -37 ]
[6/61   5/671  ----]
[              671 ]
[                  ]
[       -76    -38 ]
[7/183  ----   ----]
[       671    6039]

[       460    1195]
[1/61   ----   ----]
[       2013   6039]
[                  ]
[  17   431    631 ]
[ ---   ----   ----]
[ 183   2013   6039]
[                  ]
[              -37 ]
[6/61   5/671  ----]
[              671 ]
[                  ]
[       -76    -38 ]
[7/183  ----   ----]
[       671    6039]

# Процесс ортогонализации Грама--Шмидта

К системе из $n$ векторов применим процесс ортогонализации Грамма-Шмидта 
$$
\begin{matrix}
e_1^{new} = e_1\\
e_2^{new} = e_2 - \frac{(e_1, e_2)}{(e_1, e_1)}e_1\\
e_3^{new} = e_3 - \frac{(e_1, e_3)}{(e_1, e_1)}e_1 - \frac{(e_2, e_3)}{(e_2, e_2)}e_2\\
...\\
e_n^{new} = e_n - \frac{(e_1, e_n)}{(e_1, e_1)}e_1 - \frac{(e_2, e_n)}{(e_2, e_2)}e_2 - ... - \frac{(e_{n - 1}, e_n)}{(e_{n - 1}, e_{n - 1})}e_{n - 1}
\end{matrix}
$$
Полученные ортогональные векторы нормализуем с помощью деления на их модули.

## Пример 5
Построить ортонормированную систему векторов на основе системы
$$
\begin{matrix}
e_1 = (1, 2, 0, 1)\\
e_2 = (5, -2, 1, -1)\\
e_3 = (-1, 0, 2, -3)\\
e_4 = (0, -1, 0, 1)
\end{matrix}
$$

In [19]:
e1 = Matrix([1, 2, 0, 1])
e2 = Matrix([5, -2, 1, -1])
e3 = Matrix([-1, 0, 2, -3])
e4 = Matrix([0, -1, 0, 1])
g1 = e1
g2 = e2 - g1.dot(e2)/g1.dot(e1)*g1
g3 = e3 - g1.dot(e3)/g1.dot(g1)*g1 - g2.dot(e3)/g2.dot(g2)*g2
g4 = e4 - g1.dot(e4)/g1.dot(g1)*g1 - g2.dot(e4)/g2.dot(g2)*g2 - g3.dot(e4)/g3.dot(g3)*g3
display('normalized', *GramSchmidt([e1, e2, e3, e4], True), 
        'not normalized', *GramSchmidt([e1, e2, e3, e4]), 
        'g1234', *[sympy.simplify(item) for item in (g1, g2, g3, g4)])
print(*[sympy.simplify(item) for item in (g1.dot(g2), g1.dot(g3), g1.dot(g4), g2.dot(g3), g2.dot(g4), g3.dot(g4))])

'normalized'

[  ___]
[\/ 6 ]
[-----]
[  6  ]
[     ]
[  ___]
[\/ 6 ]
[-----]
[  3  ]
[     ]
[  0  ]
[     ]
[  ___]
[\/ 6 ]
[-----]
[  6  ]

[     ____ ]
[ 5*\/ 31  ]
[ -------- ]
[    31    ]
[          ]
[     ____ ]
[-2*\/ 31  ]
[----------]
[    31    ]
[          ]
[    ____  ]
[  \/ 31   ]
[  ------  ]
[    31    ]
[          ]
[    ____  ]
[ -\/ 31   ]
[ -------- ]
[    31    ]

[    _____  ]
[ -\/ 102   ]
[ --------- ]
[    102    ]
[           ]
[     _____ ]
[ 2*\/ 102  ]
[ --------- ]
[     51    ]
[           ]
[    _____  ]
[  \/ 102   ]
[  -------  ]
[     17    ]
[           ]
[     _____ ]
[-7*\/ 102  ]
[-----------]
[    102    ]

[     _____ ]
[-3*\/ 527  ]
[-----------]
[    527    ]
[           ]
[     _____ ]
[-5*\/ 527  ]
[-----------]
[    527    ]
[           ]
[     _____ ]
[18*\/ 527  ]
[---------- ]
[   527     ]
[           ]
[     _____ ]
[13*\/ 527  ]
[---------- ]
[   527     ]

...

[1]
[ ]
[2]
[ ]
[0]
[ ]
[1]

[5 ]
[  ]
[-2]
[  ]
[1 ]
[  ]
[-1]

[-1/3]
[    ]
[4/3 ]
[    ]
[ 2  ]
[    ]
[-7/3]

[-54 ]
[----]
[527 ]
[    ]
[-90 ]
[----]
[527 ]
[    ]
[324 ]
[--- ]
[527 ]
[    ]
[234 ]
[--- ]
[527 ]

0 0 0 0 0 0


## Пример 6
Построим с помощью QR-разложения псевдообратную матрицу для матрицы $A$, столбцы которой $e_1$, $e_2$, $e_3$ и $e_4$ из Примера 5.

Матрицу $Q$ можем сразу составить из нормированных векторов $g_1$, $g_2$, $g_3$ и $g_4$ (можно использовать GramSchmidt([e1, e2, e3, e4], True)).

Обозначим ортонормированные векторы  $f_1$, $f_2$, $f_3$ и $f_4$.

Матрица $R$ имеет вид
$$
R = \left(
\begin{matrix}
(f_1, a_1) & (f_1, a_2) & (f_1, a_3) & (f_1, a_4)\\
0 & (f_2, a_2) & (f_2, a_3) & (f_2, a_4)\\
0 & 0 & (f_3, a_3) & (f_3, a_4)\\
0 & 0 & 0 & (f_4, a_4)\\
\end{matrix}\right)
$$

Вначале составим матрицу $A$, затем получим ортонормированную систему из столбцов $A$, далее на их основе составим матрицу $Q$. Инициализируем матрицу $R$ нулевой матрицей соответствующей размерности, заполним ее в соответствии с формулой. Построим псевдообратную матрицу по формуле   $A^+ = R^T(R\cdot R^T)^{-1}Q^T$.

In [24]:
A = Matrix([e1.T, e2.T, e3.T, e4.T]).T
A

[1  5   -1  0 ]
[             ]
[2  -2  0   -1]
[             ]
[0  1   2   0 ]
[             ]
[1  -1  -3  1 ]

In [31]:
A = Matrix([e1.T, e2.T, e3.T, e4.T]).T
f = GramSchmidt([e1, e2, e3, e4], True)
Q = Matrix([fk.T for fk in f]).T
R = sympy.zeros(4, 4)
ee = (e1, e2, e3, e4)
for k in range(4):
    for j in range(k, 4):
        R[k, j] = f[k].dot(ee[j])
display('Q', Q, 'R', R, 'A', A, 'QR', Q*R, 'A+', A.pinv(), 'R.T(R*R.T).inv()Q.T', R.T*(R*R.T).inv()*Q.T)


'Q'

[  ___       ____       _____         _____ ]
[\/ 6    5*\/ 31     -\/ 102     -3*\/ 527  ]
[-----   --------    ---------   -----------]
[  6        31          102          527    ]
[                                           ]
[  ___       ____        _____        _____ ]
[\/ 6   -2*\/ 31     2*\/ 102    -5*\/ 527  ]
[-----  ----------   ---------   -----------]
[  3        31           51          527    ]
[                                           ]
[           ____        _____         _____ ]
[         \/ 31       \/ 102     18*\/ 527  ]
[  0      ------      -------    ---------- ]
[           31           17         527     ]
[                                           ]
[  ___      ____         _____        _____ ]
[\/ 6    -\/ 31     -7*\/ 102    13*\/ 527  ]
[-----   --------   -----------  ---------- ]
[  6        31          102         527     ]

'R'

[                    ___        ___    ]
[  ___          -2*\/ 6      -\/ 6     ]
[\/ 6     0     ---------    -------   ]
[                   3           6      ]
[                                      ]
[                               ____   ]
[         ____                \/ 31    ]
[  0    \/ 31       0         ------   ]
[                               31     ]
[                                      ]
[                  _____         _____ ]
[                \/ 102    -11*\/ 102  ]
[  0      0      -------   ------------]
[                   3          102     ]
[                                      ]
[                                _____ ]
[                           18*\/ 527  ]
[  0      0         0       ---------- ]
[                              527     ]

'A'

...

[1  5   -1  0 ]
[             ]
[2  -2  0   -1]
[             ]
[0  1   2   0 ]
[             ]
[1  -1  -3  1 ]

'A+'

[        11          11  ]
[1/12    --    1/2   --  ]
[        36          36  ]
[                        ]
[ 1/6   -1/18   0   -1/18]
[                        ]
[-1/12  1/36   1/2  1/36 ]
[                        ]
[                    13  ]
[-1/6   -5/18   1    --  ]
[                    18  ]

'R.T(R*R.T).inv()Q.T'

[        11          11  ]
[1/12    --    1/2   --  ]
[        36          36  ]
[                        ]
[ 1/6   -1/18   0   -1/18]
[                        ]
[-1/12  1/36   1/2  1/36 ]
[                        ]
[                    13  ]
[-1/6   -5/18   1    --  ]
[                    18  ]

# Numpy и Scipy

## LU-разложение Scipy
https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

$A = P L U$

scipy.linalg.lu(a, permute_l=False, overwrite_a=False, check_finite=True)

## QR-разложение
### Numpy
https://numpy.org/doc/stable/reference/generated/numpy.linalg.qr.html


numpy.linalg.qr(a, mode='reduced')

Пусть K = min(M, N), тогда

‘reduced’ : возвращает q, r с размерностями соответственно (M, K), (K, N) (по умолчанию)

‘complete’ : возвращает q, r с размерностями соответственно (M, M), (M, N)

‘r’ : возвращает r размерности только (K, N)

‘raw’ : возвращает h, tau с размерностями соответственно (N, M), (K,)

Значения ‘reduced’, ‘complete и ‘raw’ появились начиная с версии numpy 1.8. Значение по умолчанию ‘reduced’. Обратите внимание, что array h возвращаемый в режиме ‘raw’ транспонирован для совместимости с Fortran. Режим ‘economic’ не поддерживается больше. Режимы ‘full’ и ‘economic’ могут быть вызваны с использованием только первого символа (для обратной совместимости), но остальные пишутся полностью. 
### Scipy
https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html

scipy.linalg.qr(a, overwrite_a=False, lwork=None, mode='full', pivoting=False, check_finite=True)

При pivoting=False возвращает матрицы $Q$ и $R$. Если pivoting=True, дополнительно возвращается $P$, такая что $A P = Q R$, причем диагональные элементы $R$ образую невозрастающую последовательность.
